#### Undersampling Majority Class

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns

#### Start a simple Spark Session

In [2]:
spark = SparkSession.builder.appName('fraud_detection_undersampling').master("local[*]").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/10 20:34:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/10 20:34:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


24/06/10 20:34:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
col_names = ['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 'Use Chip', 'Merchant Name', 'Merchant City', 
'Merchant State', 'Zip', 'MCC', 'Errors?', 'Is Fraud?', 'Hour', 'Minute', 'Date', 'Day_of_Week']

df = spark.read.option("delimiter", "|").csv('df_EDA.csv/df_EDA.csv', header=None, inferSchema=True).toDF(*col_names)
df = df.repartition(10)
df = df.drop('Zip')

In [6]:
Fraud = df[df['Is Fraud?']==1]

Normal = df[df['Is Fraud?']==0]

fraud_count = Fraud.count()
normal_count = Normal.count()
print(f"fraud_count: {fraud_count} and normal_count: {normal_count}")

fraud_count: 29757 and normal_count: 24357143


In [8]:
sampling_fraction = fraud_count / normal_count
sampling_fraction

0.00122169500749739

In [9]:
# Undersample the majority class
Normal_undersampled = Normal.sample(withReplacement=False, fraction=sampling_fraction, seed=42)

# Combine the undersampled majority class with the minority class
df_balanced = Normal_undersampled.union(Fraud)
df_balanced.count()

59821

In [10]:
train, test = df_balanced.randomSplit([0.8,0.2])

In [11]:
stages = []
categorical_cols = ['Use Chip', 'Day_of_Week']
numerical_cols = ['Card', 'Year', 'Month', 'Day', 'Amount', 'MCC', 'Hour', 'Minute']

# Indexers for categorical columns
indexers = [StringIndexer(inputCol=col, outputCol=col+'_indexed') for col in categorical_cols]
# Encoders for categorical columns
encoders = [OneHotEncoder(inputCol=col+'_indexed', outputCol=col+'_OHE') for col in categorical_cols]

assembler = VectorAssembler(inputCols=(numerical_cols + ['Use Chip_OHE', 'Day_of_Week_OHE']),
                            outputCol='features')

lr_model = LogisticRegression(featuresCol='features', labelCol='Is Fraud?')

stages = indexers + encoders + [assembler, lr_model]

pipeline = Pipeline(stages=stages)

In [12]:
lr_undersampled_model = pipeline.fit(train)

24/06/10 20:43:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/10 20:43:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [13]:
predictions = lr_undersampled_model.transform(test)

In [14]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Is Fraud?')
AUC = my_eval.evaluate(predictions)
AUC

0.7649098737083811

In [15]:
evaluator = MulticlassClassificationEvaluator(labelCol="Is Fraud?", predictionCol="prediction")

# Compute metrics
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})


evaluator_precision = MulticlassClassificationEvaluator(labelCol="Is Fraud?", predictionCol="prediction", metricName="precisionByLabel")
precision_label_0 = evaluator_precision.evaluate(predictions, {evaluator_precision.metricLabel: 0.0})
precision_label_1 = evaluator_precision.evaluate(predictions, {evaluator_precision.metricLabel: 1.0})

evaluator_recall = MulticlassClassificationEvaluator(labelCol="Is Fraud?", predictionCol="prediction", metricName="recallByLabel")
recall_label_0 = evaluator_recall.evaluate(predictions, {evaluator_recall.metricLabel: 0.0})
recall_label_1 = evaluator_recall.evaluate(predictions, {evaluator_recall.metricLabel: 1.0})


f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

print(f"Precision label 1: {precision_label_1}")
print(f"Recall label 1: {recall_label_1}")

print(f"Precision label 0: {precision_label_0}")
print(f"Recall label 0: {recall_label_0}")

Accuracy: 0.7657270397619245
F1 Score: 0.7631653457133127
Precision label 1: 0.8304801670146138
Recall label 1: 0.663
Precision label 0: 0.7232790474887094
Recall label 0: 0.8668197474167624
